# Week 5 Homework

***Due (pushed to your GitHub branch) on 10/17 by 11:59 pm***

## FASTQ fetch, QC, and trimming/filtering

Use the SRA-toolkit to fetch the relevant FASTQ files. You may use a hand-crafted `txt` file to provide the list of SRAs instead of manipulating an SRA Run Table with `cut`.

In [1]:
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --split-files --sra-id $line --threads 32 --outdir hw_fastq --gzip; \
    done < hwsra_list.txt

Getting ERR1442631 from NCBI SRA
2025-10-14 21:21:31,360 - SRR ids: ['ERR1442631']
2025-10-14 21:21:31,360 - extra args: ['--split-files', '--gzip']
2025-10-14 21:21:31,364 - tempdir: /local/scratch/job_322458/pfd_tnla3xhm
2025-10-14 21:21:31,364 - CMD: sra-stat --meta --quick ERR1442631
2025-10-14 21:21:38,782 - ERR1442631 spots: 1785080
2025-10-14 21:21:38,782 - blocks: [[1, 55783], [55784, 111566], [111567, 167349], [167350, 223132], [223133, 278915], [278916, 334698], [334699, 390481], [390482, 446264], [446265, 502047], [502048, 557830], [557831, 613613], [613614, 669396], [669397, 725179], [725180, 780962], [780963, 836745], [836746, 892528], [892529, 948311], [948312, 1004094], [1004095, 1059877], [1059878, 1115660], [1115661, 1171443], [1171444, 1227226], [1227227, 1283009], [1283010, 1338792], [1338793, 1394575], [1394576, 1450358], [1450359, 1506141], [1506142, 1561924], [1561925, 1617707], [1617708, 1673490], [1673491, 1729273], [1729274, 1785080]]
2025-10-14 21:21:38,783 - 

Run QC on the FASTQ files and aggregate the results into a report.

In [5]:
!fastqc -t 32 hw_fastq/*.fastq.gz -o hw_fastq/qc
!multiqc hw_fastq/qc

[0.003s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
application/gzip
application/gzip
Started analysis of ERR1442621_1.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of ERR1442621_2.fastq.gz
Approx 5% complete for ERR1442621_1.fastq.gz
Started analysis of ERR1442622_1.fastq.gz
Approx 5% complete for ERR1442621_2.fastq.gz
Approx 10% complete for ERR1442621_1.fastq.gz
Approx 10% complete for ERR1442621_2.fastq.gz
Started analysis of ERR1442622_2.fastq.gz
Approx 15% complete for ERR1442621_1.fastq.gz
Approx 5% complete for ERR1442622_1.fastq.gz
Approx 15% complete for ERR1442621_2.fastq.gz
Approx 20% complete for ERR1442621_1.fastq.gz
Started analysis of ERR1442623_1.fast

Edit this block with a short evaluation of each of the report sections. Each evaluation should include your important observations, reasons for potential QC failure, and an opinion on what should be done on the failure (i.e., trimming or filtering). Problem FASTQ files, if any, should be noted.

*Per base sequence quality:* These are good

*Per tile sequence quality:*

*Per sequence quality scores:* These are good

*Per base sequence content:* This is bad. There is a big mess at one end of the reads. There are probably adapters remaining on that side. This will need trimming.

*Per sequence GC content:* There are 4 that seems to be a big problem (red) while others are minor problems (yellow).

*Per base N content:* This is good

*Sequence length distribution:* All samples are the same length so it is good

*Sequence duplication levels:* Most of the library is not duplicate which is good

*Overrepresented sequences:* 7 samples have somewhat overrepresented sequences (yellow).

*Adapter content:* this is good as well.

If necessary, use `cutadapt` to trim and filter your reads. Provide a justification for the `cutadapt` parameters you used:



In [6]:
!for fastq in hw_fastq/*.fastq.gz; do \
    base_name=$(basename "$fastq" .fastq.gz); \
    cutadapt -j 32 -u 15 -m 85 -o ./hwtrimmed/$base_name.fastq.gz $fastq; \
done

This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -u 15 -m 85 -o ./hwtrimmed/ERR1442621_1.fastq.gz hw_fastq/ERR1442621_1.fastq.gz
Processing single-end reads on 32 cores ...
Done           00:00:01     1,706,411 reads @   1.1 µs/read;  56.81 M reads/minute
Finished in 2.087 s (1.223 µs/read; 49.07 M reads/minute).

=== Summary ===

Total reads processed:               1,706,411

== Read fate breakdown ==
Reads that were too short:                   0 (0.0%)
Reads written (passing filters):     1,706,411 (100.0%)

Total basepairs processed:   170,641,100 bp
Total written (filtered):    145,044,935 bp (85.0%)
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -u 15 -m 85 -o ./hwtrimmed/ERR1442621_2.fastq.gz hw_fastq/ERR1442621_2.fastq.gz
Processing single-end reads on 32 cores ...
Done           00:00:01     1,706,411 reads @   0.9 µs/read;  64.91 M reads/minute
Finished in 1.689 s (0.990 µs/read; 60.61 M reads/minute).

=== Summary ===

Total rea

Run QC on the trimmed/filtered reads and generate an aggregated report

In [7]:
!fastqc -t 32 hwtrimmed/*.fastq.gz -o hwtrimmed/qc
!multiqc --force -d hw_fastq/qc/ hwtrimmed/qc/

[0.001s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
application/gzip
application/gzip
Started analysis of ERR1442621_1.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of ERR1442621_2.fastq.gz
Approx 5% complete for ERR1442621_1.fastq.gz
Approx 5% complete for ERR1442621_2.fastq.gz
Started analysis of ERR1442622_1.fastq.gz
Approx 10% complete for ERR1442621_1.fastq.gz
Approx 10% complete for ERR1442621_2.fastq.gz
Approx 15% complete for ERR1442621_1.fastq.gz
Approx 5% complete for ERR1442622_1.fastq.gz
Started analysis of ERR1442622_2.fastq.gz
Approx 15% complete for ERR1442621_2.fastq.gz
Approx 20% complete for ERR1442621_1.fastq.gz
Approx 10% complete for ERR1442622_1.

Report your observations on which QC metrics improved (or got worse): all red fails disappeared from "Per Base Sequence Content" and "Per Sequence GC Content"



## Snakemake Pipeline

When running bioinformatics studies, it is wise to use notebooks (such as this one) to tie together code, decisions, and data visualization. The above task is something you might generate and report to a client or supervisor who might be interested in sequencing QC and what kind of pre-processing decisions you made prior to alignment. In addition, the preprocessing should be readily reproducible by all relevant stakeholders, so you should supplement notebooks with pipeline scripts. Throughout the semester, we will be creating snakemake pipelines to reproduce the published analysis that you've selected. Follow the below steps to begin producing this pipeline:

1. Make a new `Snakefile` and `config.yaml` in `5_fastq`. 
2. In the `Snakefile`, produce the following rules:  
    a. `fetch_fastq` - use `parallel-fastq-dump` to fetch every FASTQ file listed in `config.yaml`.  
    b. `fastq_qc` - use `fastqc` to run QC on the raw FASTQ files.  
    c. `trim_filter` - use `cutadapt` to trim and/or filter the FASTQ reads using parameters justified in this notebook. ***Note:*** if you choose to trim/filter, it should be done on *all* of the FASTQ files; do not cherry pick.  
    d. `trim_qc` - use `fastqc` to run QC on the trimmed FASTQ files.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`. You may use the documentation found [here](https://multiqc.info/docs/usage/pipelines/#snakemake) or any other online documentation you may find. You may not use any AI tools to complete this homework.